In [ ]:
## IMPORTANT: This notebook does not work anymore since the super-mario-bros package is unmaintained

import random
import json
import numpy
import matplotlib.pyplot as plt
import torch
import gymnasium as gym
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

torch.set_default_device("cuda")

/home/spexx/Desktop/ssd/Code/ArtI/RL/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:
class ReplayBuffer:
    def __init__(self, maxLen):
        self.Buffer = deque([], maxlen=maxLen)

    def add(self,data):
        self.Buffer.append(data)
        return
    
    def sample(self, batchSize):
        # returns a list with the batches as a list at each index
        x = random.sample(self.Buffer, batchSize)
        return list(zip(*x))
    
    def altSample(self, batchSize):
        # might work, might not
        indices = random.sample(range(len(self.Buffer)), batchSize)
        batch = [[self.Buffer[x][y] for x in indices] for y in range(len(self.Buffer[0]))]
        return batch

In [3]:
class Network(torch.nn.Module):
    def __init__(self, inFrames, outputDimension):
        super().__init__()
        # initializes layers with kaiming uniform
        self.Conv2d_1 = torch.nn.Conv2d(in_channels=inFrames, out_channels=32, kernel_size=8, stride=4)
        self.Conv2d_2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2)
        self.Conv2d_3 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.Linear_1 = torch.nn.Linear(3136, 512) 
        self.Linear_2 = torch.nn.Linear(512, outputDimension)

    def forward(self, input):
        # input (1,84,84)
        x = self.Conv2d_1(input)
        x = torch.nn.functional.relu(x)
        x = self.Conv2d_2(x)
        x = torch.nn.functional.relu(x)
        x = self.Conv2d_3(x)
        x = torch.nn.functional.relu(x)
        x = x.flatten()
        x = self.Linear_1(x)
        x = torch.nn.functional.relu(x)
        x = self.Linear_2(x)
        # output (outputDimension)
        return x
    
    def forwardBatch(self, input):
        # input (Batch,1,84,84)
        x = self.Conv2d_1(input)
        x = torch.nn.functional.relu(x)
        x = self.Conv2d_2(x)
        x = torch.nn.functional.relu(x)
        x = self.Conv2d_3(x)
        x = torch.nn.functional.relu(x)
        x = x.flatten(start_dim = 1)
        x = self.Linear_1(x)
        x = torch.nn.functional.relu(x)
        x = self.Linear_2(x)
        # output (Batch, outputDimension)
        return x

In [4]:
def pickAction(state):
    global Epsilon

    if(Epsilon > EpsilonEnd):
        Epsilon = Epsilon - ((EpsilonStart-EpsilonEnd)/EpsilonStepsbetweenStartandEnd)
    if(random.random()<Epsilon):
        return env.action_space.sample()
    else:
        prediction = MarioPlayer(state)
        average_q.append(torch.max(prediction).item())
        return torch.argmax(prediction).item()

In [5]:
def FrameskipStep(env, action, SkipFrames):
    fullReward = 0
    for _ in range(SkipFrames):
        observation, reward, terminated, truncated, info = env.step(action)
        fullReward += reward
        if(terminated or truncated):
            return observation, fullReward, terminated, truncated, info
    return observation, fullReward, terminated, truncated, info

In [6]:
def Visualization(StateBatch, actionBatch, rewardBatch, NextStateBatch, predictions, nextStateprediction, target):
    # input / output vizualization
    # state
    fig, ax = plt.subplots(1, inFrames, figsize=(13, 10))
    for i in range(inFrames):
        ax[i].imshow(StateBatch[0][i].cpu(), cmap='gray', vmin=0, vmax=255)
        plt.tight_layout()
    plt.show()
    # predictions for the moves
    print("Predictions for the moves:\nNOOP, RIGHT, RIGHT + JUMP, RIGHT + RUN, RIGHT + RUN + JUMP, JUMP, LEFT")
    print(predictions[0].cpu())
    # actual move
    print("Actual move (might be different from highest predicted value because decision was made by old model or randomly)")
    print(actionBatch[0].cpu())
    # nextState
    fig, ax = plt.subplots(1, inFrames, figsize=(13, 10))
    for i in range(inFrames):
        plt.imshow(NextStateBatch[0][i].cpu(), cmap='gray', vmin=0, vmax=255)
        ax[i].imshow(NextStateBatch[0][i].cpu(), cmap='gray', vmin=0, vmax=255)
        plt.tight_layout()
    plt.show()
    # nextStatePredictions
    print("Prediction for the moves in the next State:\nNOOP, RIGHT, RIGHT + JUMP, RIGHT + RUN, RIGHT + RUN + JUMP, JUMP, LEFT")
    print(nextStateprediction[0].cpu())
    # actual reward
    print("Actual reward")
    print(rewardBatch[0])
    # target
    print("Target (reward + gamma*Highest predicted action value in NextState)")
    print(target[0])

In [7]:
def Optimizer(MarioPlayer, MarioBuffer, BatchSize, optimizer, lossFunction):
    # TD Learning
    if (len(MarioBuffer.Buffer)<BatchSize):
        return
    Batches = MarioBuffer.sample(BatchSize)
    StateBatch = torch.cat(Batches[0])
    actionBatch = torch.cat(Batches[1])
    rewardBatch = torch.cat(Batches[2])
    NextStateBatch = torch.cat(Batches[3])
    terminated_batch = torch.cat(Batches[4])

    # zeroes the gradients because default behaviour in PT is to accumulate them
    for param in MarioPlayer.parameters():
        param.grad = None

    predictions = MarioPlayer.forwardBatch(StateBatch)
    prediction = predictions.gather(1, actionBatch)

    with torch.no_grad():
        nextStateprediction = MarioPredictor.forwardBatch(NextStateBatch)
        nextStatepredictionMax = nextStateprediction.max(1).values.unsqueeze(1)
        target = rewardBatch + gamma*(nextStatepredictionMax*terminated_batch)

    if(i%10000 == 0):
        print(f"{i}\n")
        Visualization(StateBatch, actionBatch, rewardBatch, NextStateBatch, predictions, nextStateprediction, target)

    loss = lossFunction(prediction,target)
    if(i%100 == 0):
        writer.add_scalar("Loss/Batch", loss.item(), global_step=i)
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(MarioPlayer.parameters(), 10)
    optimizer.step()
    
    return 

In [8]:
# hyperparameters

EpsilonStart = 0.1
Epsilon = EpsilonStart
EpsilonStepsbetweenStartandEnd = 1_000_000
EpsilonEnd = 0.1

gamma = 0.99
BatchSize = 32

inFrames = 4
possibleActions = 7
# the agent only sees every nth frame (1, 1+n, 1+2n, etc.) so the agent doesnt see the n-1 frames in between
SkipFrames = 6
# 1 means optimization every step, 2 every 2nd step etc.
steps_per_optimization = 1
steps_per_target_net_update = 10_000


BufferLength = 100_000

agent = "Mario_DQN_3"

logdir = f"./logs/{agent}"

MarioPlayer = Network(inFrames,possibleActions)
# MarioPlayer.load_state_dict(torch.load(f"./models/{agent}", weights_only=True))

MarioPredictor = Network(inFrames,possibleActions)
MarioPredictor.load_state_dict(MarioPlayer.state_dict())

MarioBuffer = ReplayBuffer(BufferLength)

optimizer = torch.optim.Adam(MarioPlayer.parameters(), lr=1e-4)

# what reduction to use? if you sum the gradients are bigger (also depend on batchSize then)
# just use mean for now
lossFunction = torch.nn.SmoothL1Loss()

trainingSteps = 10_001

In [10]:
# logging setup

writer = SummaryWriter(log_dir=logdir)

# sampling trajectories loop

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = gym.wrappers.ResizeObservation(env, (84,84))
env = gym.wrappers.GrayScaleObservation(env, keep_dim = True)
env = gym.wrappers.FrameStack(env, 4)

observation, info = env.reset()
for _ in range(random.randint(0,30)):
    observation, reward, terminated, truncated, info = env.step(0)    

nextState = torch.tensor(numpy.array(observation), dtype=torch.float32)
nextState= nextState.reshape((1,4,84,84))

episodeReward = 0
average_q = []

for i in range(trainingSteps):
    state = nextState
    action = pickAction(state)
    observation, reward, terminated, truncated, info = FrameskipStep(env, action, SkipFrames)
    if(info['flag_get'] == True):
        reward += 15
    episodeReward += reward
    nextState = torch.tensor(numpy.array(observation), dtype=torch.float32)
    nextState = nextState.reshape((1,4,84,84))

    MarioBuffer.add([state,torch.tensor([[action]]),torch.tensor([[reward]]),nextState, torch.tensor([[not (terminated or truncated)]])])

    if(i % steps_per_optimization == 0):
        Optimizer(MarioPlayer, MarioBuffer, BatchSize, optimizer, lossFunction)

    if(i % steps_per_target_net_update == 0):
        MarioPredictor.load_state_dict(MarioPlayer.state_dict())

    if terminated or truncated:
        observation, info = env.reset() 

        for _ in range(random.randint(0,30)):
            observation, reward, terminated, truncated, info = env.step(0)  

        nextState = torch.tensor(numpy.array(observation), dtype=torch.float32)
        nextState = nextState.reshape((1,4,84,84))

        writer.add_scalar("Reward/Episode", episodeReward, global_step=i)

        episodeReward = 0
        average_q_scalar = 0

        if(len(average_q) > 0):
            for j in average_q:
                average_q_scalar += j
            average_q_scalar /= len(average_q)
        writer.add_scalar("Average_Q", average_q_scalar, global_step=i)
        average_q = []

    if (i%100000 == 0):
        torch.save(MarioPlayer.state_dict(), f"./models/{agent}_{i}")    

env.close()

# save model
torch.save(MarioPlayer.state_dict(), f"./models/{agent}")

writer.flush()
writer.close()

/home/spexx/Desktop/ssd/Code/ArtI/RL/.venv/lib/python3.11/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3` with the environment ID `SuperMarioBros-v3`.
  logger.warn(


OverflowError: Python integer 1024 out of bounds for uint8

In [ ]:
Epsilon = 0.05

inFrames = 4
possibleActions = 7
# the agent only sees every nth frame (1, 1+n, 1+2n, etc.) so the agent doesnt see the n-1 frames in between
SkipFrames = 6

agent = "Mario_DQN_2"

logdir = f"logs/{agent}"

MarioPlayer = Network(inFrames,possibleActions)
MarioPlayer.load_state_dict(torch.load(f"./models/{agent}", weights_only=True))

In [ ]:
# validation enviroment

env = gym_super_mario_bros.make('SuperMarioBros-1-4-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = gym.wrappers.ResizeObservation(env, (84,84))
env = gym.wrappers.GrayScaleObservation(env, keep_dim = True)
env = gym.wrappers.FrameStack(env, 4)

observation, info = env.reset()
for _ in range(random.randint(0,30)):
    observation, reward, terminated, truncated, info = env.step(0)  

validationEpisodeReward = 0

with torch.no_grad():
    observation, info = env.reset()
    nextState = torch.tensor(numpy.array(observation), dtype=torch.float32)
    nextState= nextState.reshape((1,4,84,84))

    terminated, truncated = False, False

    while not terminated and not truncated:
        state = nextState
        action = pickAction(state)
        observation, reward, terminated, truncated, info = FrameskipStep(env, action, SkipFrames)
        env.render()
        time.sleep(0.05)
        validationEpisodeReward += reward
        nextState = torch.tensor(numpy.array(observation), dtype=torch.float32)
        nextState = nextState.reshape((1,4,84,84))

print(validationEpisodeReward)

env.close()